# Simulation of Pond System using Python-Control

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from control import input_output_response, NonlinearIOSystem

import control
control.__version__

'0.9.1'

In [16]:
from platform import python_version
python_version()

'3.9.12'

In [2]:
from pond_model_eqns import f, g, params, u_nop, x_nop

In [6]:
help(f)

Help on function f in module pond_model_eqns:

f(t, x, u, params)
    Continuous-time dynamics



In [7]:
help(g)

Help on function g in module pond_model_eqns:

g(t, x, u, params)
    Measurement function



In [17]:
sys = NonlinearIOSystem(f, outfcn=g, params=params, inputs=('Flow in (m^3/s)'), 
                        outputs=('Head on weir (m^3/s)'), name='Pond flow model')
print(sys)

System: Pond flow model
Inputs (1): Flow in (m^3/s), 
Outputs (1): Head on weir (m^3/s), 
States (None): 


In [23]:
T = np.linspace(0, 5*60, 51)
U = np.zeros(T.shape)
U[t > 60] = 1
assert(U.shape == t.shape)
sim_out = input_output_response(sys, T, U, params=params)

ValueError: could not broadcast input array from shape (0,) into shape (1,)

In [14]:
U.shape

(51,)

In [ ]:
help(input_output_response)